# Compare Transit Skims

In [3]:
import os
import pandas as pd
import numpy as np
import openmatrix as omx 

### Remote I/O

In [57]:
legacy_dir = os.path.join("C:/", "projects", "met_council", "model_files", "outputs2015", "outputs")
new_dir = os.path.join("D:/", "david_ory_github", "client_met_council_network", "data", "test_new_network_outputs")
compare_dir = os.path.join("D:/", "david_ory_github", "client_met_council_network", "data", "interim")

ivtt_filename = os.path.join(compare_dir, "compare-ivtt.csv")

### Start with walk transit

In [9]:
filename = os.path.join(legacy_dir, "XIT_WK_SKIM_4_PK.OMX")
legacy_omx = omx.open_file(filename)

filename = os.path.join(new_dir, "XIT_WK_SKIM_0_PK.OMX")
new_omx = omx.open_file(filename)

In [10]:
legacy_omx.list_matrices()

['FARE',
 'FAREP',
 'FARER',
 'FARES',
 'IWAIT',
 'TRNTIME',
 'WLKACC',
 'WLKEGR',
 'WLKXFER',
 'XFERS',
 'XWAIT']

In [11]:
new_omx.list_matrices()

['FARE',
 'FAREP',
 'FARER',
 'FARES',
 'IWAIT',
 'TRNTIME',
 'WLKACC',
 'WLKEGR',
 'WLKXFER',
 'XFERS',
 'XWAIT']

In [42]:
def make_dataframe(input_mtx, column_name):
    df = pd.DataFrame(input_mtx)
    df = df.add_prefix('dest_')
    df['orig'] = df.index
    df = pd.wide_to_long(df, 'dest_', 'orig', 'dest')
    df = df.reset_index().rename(columns = {'dest_': column_name})
    df['orig'] = df['orig'] + 1
    df['dest'] = df['dest'] + 1
    
    return df

In [49]:
leg_df = make_dataframe(legacy_omx['TRNTIME'], 'legacy')

In [50]:
new_df = make_dataframe(new_omx['TRNTIME'], 'new')

In [51]:
transit_df = pd.merge(
    leg_df,
    new_df,
    how = 'outer',
    on = ['orig', 'dest']
)

In [52]:
transit_df.head()

,orig,dest,legacy,new
0,1,1,0.0,0.0
1,2,1,0.0,0.0
2,3,1,0.0,0.0
3,4,1,0.0,0.0
4,5,1,0.0,0.0


In [54]:
out_df = transit_df[(transit_df['legacy'] !=0) | (transit_df['new'] !=0)]

In [55]:
out_df.shape

(3289770, 4)

In [56]:
transit_df.shape

(9369721, 4)

In [59]:
out_df.to_csv(ivtt_filename, index = False)